# Q-learning-State Value Function

DQN(Deep Q-Network)：结合了 Value Function Approximation（价值函数近似）与神经网络技术，并采用了目标网络（Target Network）和经历回放（Experience Replay）的方法进行网络的训练。

State-value Function:本质是一种critic。

State-value Function Bellman Equation：表示在状态 $s_t$ 下带来的累积奖励 $G_t$ 的期望。

**Target Network：** 为了解决在基于TD的Network的问题时，优化目标 $\mathrm{Q}^{\pi}\left(s_{t}, a_{t}\right) 
=r_{t}+\mathrm{Q}^{\pi}\left(s_{t+1}, \pi\left(s_{t+1}\right)\right)$ 左右两侧会同时变化使得训练过程不稳定，从而增大regression的难度。target network选择将上式的右部分即 $r_{t}+\mathrm{Q}^{\pi}\left(s_{t+1}, \pi\left(s_{t+1}\right)\right)$ 固定，通过改变上式左部分的network的参数，进行regression。也是一个DQN中比较重要的tip。

**Exploration：**  在我们使用Q-function的时候，我们的policy完全取决于Q-function，有可能导致出现对应的action是固定的某几个数值的情况，而不像policy gradient中的output为随机的，我们再从随机的distribution中sample选择action。这样会导致我们继续训练的input的值一样，从而”加重“output的固定性，导致整个模型的表达能力的急剧下降，这也就是`探索-利用窘境(Exploration-Exploitation dilemma)`问题。所以我们使用`Epsilon Greedy`和 `Boltzmann Exploration`等Exploration方法进行优化。

**Experience Replay（经验回放）：**  其会构建一个Replay Buffer（Replay Memory），用来保存许多data，每一个data的形式如下：在某一个 state $s_t$，采取某一个action $a_t$，得到了 reward $r_t$，然后跳到 state $s_{t+1}$。我们使用 $\pi$ 去跟环境互动很多次，把收集到的数据都放到这个 replay buffer 中。当我们的buffer”装满“后，就会自动删去最早进入buffer的data。在训练时，对于每一轮迭代都有相对应的batch（与我们训练普通的Network一样通过sample得到），然后用这个batch中的data去update我们的Q-function。综上，Q-function再sample和训练的时候，会用到过去的经验数据，所以这里称这个方法为Experience Replay，其也是DQN中比较重要的tip。

## 为什么在DQN中采用价值函数近似（Value Function Approximation）的表示方法？

因为逼近真正的value function比较难，维度灾难、数量巨大

## 我们通常怎么衡量state value function  $V^{\pi}(s)$ ?分别的优势和劣势有哪些？

MC：随机性较大，方差大，但是如果能sample所有可能较准确些

TD：只要在游戏的某一个情况，某一个 state $s_t$ 的时候，采取 action $a_t$ 得到 reward $r_t$ ，跳到 state $s_{t+1}$，就可以应用 TD 的方法。公式与之前介绍的TD方法类似，即：$V^{\pi}\left(s_{t}\right)=V^{\pi}\left(s_{t+1}\right)+r_{t}$



对于TD来说，$r$ 是一个 random 变量。但是相对于MC的 $G_a$  的随机程度来说， $r$ 的随机性非常小，这是因为本身 $G_a$ 就是由很多的 $r$ 组合而成的。但另一个角度来说， 在TD中，我们的前提是 $r_t=V^{\pi}\left(s_{t+1}\right)-V^{\pi}\left(s_{t}\right)$ ,但是我们通常无法保证 $V^{\pi}\left(s_{t+1}\right)、V^{\pi}\left(s_{t}\right)$ 计算的误差为零。所以当 $V^{\pi}\left(s_{t+1}\right)、V^{\pi}\left(s_{t}\right)$  计算的不准确的话，那应用上式得到的结果，其实也会是不准的。所以 MC 跟 TD各有优劣。

## 基于上面介绍的基于TD的network方法，具体地，我们应该怎么训练模型呢？

核心的函数为 $V^{\pi}\left(s_{t}\right)=V^{\pi}\left(s_{t+1}\right)+r_{t}$。我们将state $s_t$  作为input输入network 里，因为 $s_t$ 丢到 network 里面会得到output $V^{\pi}(s_t)$，同样将 $s_{t+1}$ 作为input输入 network 里面会得到$V^{\pi}(s_{t+1})$。同时核心函数：$V^{\pi}\left(s_{t}\right)=V^{\pi}\left(s_{t+1}\right)+r_{t}$  告诉我们，  $V^{\pi}(s_t)$ 减 $V^{\pi}(s_{t+1})$ 的值应该是 $r_t$。然后希望它们两个相减的 loss 跟 $r_t$ 尽可能地接近。这也就是我们这个network的优化目标或者说loss function。

## 当我们有了Q-function后，我们怎么找到更好的策略 $\pi'$ 呢？或者说这个 $\pi'$ 本质来说是什么？

首先， $\pi'$ 是由 $\pi^{\prime}(s)=\arg \max _{a} Q^{\pi}(s, a)$ 计算而得，其表示假设你已经 learn 出 $\pi$ 的Q-function，今天在某一个 state s，把所有可能的 action a 都一一带入这个 Q-function，看看说那一个 a 可以让 Q-function 的 value 最大，那这一个 action，就是 $\pi'$ 会采取的 action。所以根据上式决定的actoin的步骤一定比原来的 $\pi$ 要好，即$V^{\pi^{\prime}}(s) \geq V^{\pi}(s)$。

## 解决`探索-利用窘境(Exploration-Exploitation dilemma)`问题的Exploration的方法有哪些？他们具体的方法是怎样的？

1. **Epsilon Greedy：** 我们有$1-\varepsilon$ 的机率，通常 $\varepsilon$ 很小，完全按照Q-function 来决定action。但是有 $\varepsilon$ 的机率是随机的。通常在实现上 $\varepsilon$ 会随着时间递减。也就是在最开始的时候。因为还不知道那个action 是比较好的，所以你会花比较大的力气在做 exploration。接下来随着training 的次数越来越多。已经比较确定说哪一个Q 是比较好的。你就会减少你的exploration，你会把 $\varepsilon$ 的值变小，主要根据Q-function 来决定你的action，比较少做random，这是**Epsilon Greedy**。
2. **Boltzmann Exploration：** 这个方法就比较像是 policy gradient。在 policy gradient 里面network 的output 是一个 expected action space 上面的一个的 probability distribution。再根据 probability distribution 去做 sample。所以也可以根据Q value 去定一个 probability distribution，假设某一个 action 的 Q value 越大，代表它越好，我们采取这个 action 的机率就越高。这是**Boltzmann Exploration**。

## 我们使用Experience Replay（经验回放）有什么好处？

1. 首先，在强化学习的整个过程中， 最花时间的 step 是在跟环境做互动，使用GPU乃至TPU加速来训练 network 相对来说是比较快的。而用 replay buffer 可以减少跟环境做互动的次数，因为在训练的时候，我们的 experience 不需要通通来自于某一个policy（或者当前时刻的policy）。一些过去的 policy 所得到的 experience 可以放在 buffer 里面被使用很多次，被反复的再利用，这样让你的 sample 到 experience 的利用是高效的。
2. 另外，在训练网络的时候，其实我们希望一个 batch 里面的 data 越 diverse 越好。如果你的 batch 里面的 data 都是同样性质的，我们的训练出的模型拟合能力不会很乐观。如果 batch 里面都是一样的 data，你 train 的时候，performance 会比较差。我们希望 batch data 越 diverse 越好。那如果 buffer 里面的那些 experience 通通来自于不同的 policy ，那你 sample 到的一个 batch 里面的 data 会是比较 diverse 。这样可以保证我们模型的性能至少不会很差。

## 在Experience Replay中我们是要观察 $\pi$ 的 value，里面混杂了一些不是 $\pi$ 的 experience ，这会有影响吗？

没关系。这并不是因为过去的 $\pi$ 跟现在的 $\pi$ 很像， 就算过去的$\pi$ 没有很像，其实也是没有关系的。主要的原因是我们并不是去sample 一个trajectory，我们只sample 了一个experience，所以跟是不是 off-policy 这件事是没有关系的。就算是off-policy，就算是这些 experience 不是来自于 $\pi$，我们其实还是可以拿这些 experience 来估测 $Q^{\pi}(s,a)$。

# Q-learning-Double DQN

Double DQN:可以避免 over estimate 的问题。具体来说，假设我们有两个 Q-function，假设第一个Q-function 它高估了它现在选出来的action a，那没关系，只要第二个Q-function $Q'$ 没有高估这个action a 的值，那你算出来的，就还是正常的值。

Dueling DQN:将原来的DQN的计算过程分为**两个path**。对于第一个path，会计算一个于input state有关的一个标量 $V(s)$；对于第二个path，会计算出一个vector $A(s,a)$ ，其对应每一个action。最后的网络是将两个path的结果相加，得到我们最终需要的Q value。用一个公式表示也就是 $Q(s,a)=V(s)+A(s,a)$ 。

**Prioritized Experience Replay （优先经验回放）：** 这个方法是为了解决我们在chapter6中提出的**Experience Replay（经验回放）**方法不足进一步优化提出的。我们在使用Experience Replay时是uniformly取出的experience buffer中的sample data，这里并没有考虑数据间的权重大小。例如，我们应该将那些train的效果不好的data对应的权重加大，即其应该有更大的概率被sample到。综上， prioritized experience replay 不仅改变了 sample data 的 distribution，还改变了 training process。

**Noisy Net：** 其在每一个episode 开始的时候，即要和环境互动的时候，将原来的Q-function 的每一个参数上面加上一个Gaussian noise。那你就把原来的Q-function 变成$\tilde{Q}$ ，即**Noisy Q-function**。同样的我们把每一个network的权重等参数都加上一个Gaussian noise，就得到一个新的network $\tilde{Q}$。我们会使用这个新的network从与环境互动开始到互动结束。

$\tilde{Q}$ ，将原来的Q-function 的每一个参数上面加上一个Gaussian noise。即**Noisy Q-function**。同样的我们把每一个network的权重等参数都加上一个Gaussian noise，就得到一个新的network $\tilde{Q}$。我们会使用这个新的network从与环境互动开始到互动结束。

**Distributional Q-function：** 对于DQN进行model distribution。将最终的网络的output的每一类别的action再进行distribution。

**Rainbow：** 也就是将我们这两节内容所有的七个tips综合起来的方法，7个方法分别包括：DQN、DDQN、Prioritized DDQN、Dueling DDQN、A3C、Distributional DQN、Noisy DQN，进而考察每一个方法的贡献度或者是否对于与环境的交互式正反馈的。

## 为什么传统的DQN的效果并不好？参考公式 $Q(s_t ,a_t)=r_t+\max_{a}Q(s_{t+1},a)$ 

1.$r_t$具有随机性

2.因为max最大的Qvalue这里较容易高估

## 我们应该怎么解决target 总是太大的问题呢？

Double DQN

## 哪来 Q  跟 $Q'$ 呢？哪来两个 network 呢？

在实现上，你有两个 Q-network， 一个是 target 的 Q-network，一个是真正你会 update 的 Q-network。所以在 Double DQN 里面，你的实现方法会是拿你会 update 参数的那个 Q-network 去选action，然后你拿target 的network，那个固定住不动的network 去算value。而 Double DQN 相较于原来的 DQN 的更改是最少的，它几乎没有增加任何的运算量，连新的network 都不用，因为你原来就有两个network 了。你唯一要做的事情只有，本来你在找最大的a 的时候，你在决定这个a 要放哪一个的时候，你是用$Q'$ 来算，你是用target network 来算，现在改成用另外一个会 update 的 Q-network 来算。

## 如何理解Dueling DQN的模型变化带来的好处？

对于我们的 $Q(s,a)$ 其对应的state由于为table的形式，所以是离散的，而实际中的state不是离散的。对于 $Q(s,a)$ 的计算公式， $Q(s,a)=V(s)+A(s,a)$ 。其中的 $V(s)$ 是对于不同的state都有值，对于 $A(s,a)$ 对于不同的state都有不同的action对应的值。所以本质上来说，我们最终的矩阵 $Q(s,a)$ 的结果是将每一个 $V(s)$ 加到矩阵 $A(s,a)$ 中得到的。从模型的角度考虑，我们的network直接改变的 $Q(s,a)$ 而是 更改的 $V、A$ 。但是有时我们update时不一定会将 $V(s)$ 和 $Q(s,a)$ 都更新。我们将其分成两个path后，我们就不需要将所有的state-action pair都sample一遍，我们可以使用更高效的estimate Q value方法将最终的 $Q(s,a)$ 计算出来。

## 使用MC和TD平衡方法的优劣分别有哪些？

- 优势：因为我们现在 sample 了比较多的step，之前是只sample 了一个step， 所以某一个step 得到的data 是真实值，接下来都是Q value 估测出来的。现在sample 比较多step，sample N 个step 才估测value，所以估测的部分所造成的影响就会比小。
- 劣势：因为我们的 reward 比较多，当我们把 N 步的 reward 加起来，对应的 variance 就会比较大。但是我们可以选择通过调整 N 值，去在variance 跟不精确的 Q 之间取得一个平衡。这里介绍的参数 N 就是一个hyper parameter，你要调这个N 到底是多少，你是要多 sample 三步，还是多 sample 五步。

# Q-learning for Continuous Actions

## Q-learning相比于policy gradient based方法为什么训练起来效果更好，更平稳？

在 Q-learning 中，只要能够 estimate 出Q-function，就可以保证找到一个比较好的 policy，同样的只要能够 estimate 出 Q-function，就保证可以 improve 对应的 policy。而因为 estimate Q-function 作为一个回归问题，是比较容易的。在这个回归问题中， 我们可以时刻观察我们的模型训练的效果是不是越来越好，一般情况下我们只需要关注 regression 的 loss 有没有下降，你就知道你的 model learn 的好不好。所以 estimate Q-function 相较于 learn 一个 policy 是比较容易的。你只要 estimate Q-function，就可以保证说现在一定会得到比较好的 policy，同样其也比较容易操作。

## Q-learning在处理continuous action时存在什么样的问题呢？

action是continuous的情况下，我们就不能像离散的action一样，穷举所有可能的continuous action了

为了解决这个问题，有以下几种solutions：

- 第一个解决方法：我们可以使用所谓的sample方法，即随机sample出N个可能的action，然后一个一个带到我们的Q-function中，计算对应的N个Q value比较哪一个的值最大。但是这个方法因为是sample所以不会非常的精确。
- 第二个解决方法：我们将这个continuous action问题，看为一个优化问题，从而自然而然地想到了可以用gradient ascend去最大化我们的目标函数。具体地，我们将action看为我们的变量，使用gradient ascend方法去update action对应的Q-value。但是这个方法通常的时间花销比较大，因为是需要迭代运算的。
- 第三个解决方法：设计一个特别的network架构，设计一个特别的Q-function，使得解我们 argmax Q-value的问题变得非常容易。也就是这边的 Q-function 不是一个 general 的 Q-function，特别设计一下它的样子，让你要找让这个 Q-function 最大的 a 的时候非常容易。但是这个方法的function不能随意乱设，其必须有一些额外的限制。
- 第四个解决方法：不用Q-learning，毕竟用其处理continuous的action比较麻烦。

### solution3


* Input 这个 s，Q-function 会 output 3 个东西。它会 output $\mu(s)$，这是一个 vector。它会 output $\Sigma(s)$ ，这是一个 matrix。它会 output $V(s)$，这是一个 scalar。
* output 这 3 个东西以后，我们知道 Q-function 其实是吃一个 s 跟 a，然后决定一个 value。Q-function 意思是说在某一个 state，take 某一个 action 的时候，你 expected 的 reward 有多大。到目前为止这个 Q-function 只吃 s，它还没有吃 a 进来，a 在哪里呢？当这个 Q-function 吐出 $\mu$、 $\Sigma$ 跟 $V$ 的时候，我们才把 a 引入，用 a 跟 $\mu(s)、\Sigma(s)、V$  互相作用一下，你才算出最终的 Q value。
* a 怎么和这 3 个东西互相作用呢？实际上 $Q(s,a)$，你的 Q-function 的运作方式是先 input s，让你得到 $\mu,\Sigma$ 跟 V。然后再 input a，然后接下来把 a 跟 $\mu$ 相减。注意一下 a 现在是 continuous 的 action，所以它也是一个 vector。假设你现在是要操作机器人的话，这个 vector 的每一个 dimension，可能就对应到机器人的某一个关节，它的数值就是关节的角度，所以 a 是一个 vector。把 a 的这个 vector 减掉 $\mu$ 的这个 vector，取 transpose，所以它是一个横的 vector。$\Sigma$ 是一个 matrix。然后 a 减掉 $\mu(s)$ ，a 和 $\mu(s)$ 都是 vector，减掉以后还是一个竖的 vector。所以 $-(a-\mu(s))^{T} \Sigma(s)(a-\mu(s))+V(s)$ 是一个 scalar，这个数值就是 Q value $Q(s,a)$，。
* 假设 $Q(s,a)$ 定义成这个样子，我们要怎么找到一个 a 去 maximize 这个 Q value 呢？这个 solution 非常简单，什么样的 a， 可以让这一个 Q-function 最终的值最大呢？。因为 $(a-\mu(s))^{T} \Sigma(s)(a-\mu(s))$ 一定是正的，它前面乘上一个负号，所以第一项就假设我们不看这个负号的话，第一项的值越小，最终的 Q value 就越大。因为我们是把 V(s) 减掉第一项，所以第一项的值越小，最后的 Q value 就越大。怎么让第一项的值最小呢？你直接把 a 代入 $\mu$ 的值，让它变成 0，就会让第一项的值最小。
* $\Sigma$ 一定是正定的。因为这个东西就像是 Gaussian distribution，所以 $\mu$ 就是 Gaussian 的 mean，$\Sigma$ 就是 Gaussian 的 variance。但 variance 是一个 positive definite 的 matrix，怎么样让这个 $\Sigma$ 一定是 positive definite 的 matrix 呢？其实在 $Q^{\pi}$ 里面，它不是直接 output $\Sigma$，如果直接 output 一个 $\Sigma$， 它不一定是 positive definite 的 matrix。它其实是 output 一个 matrix，然后再把那个 matrix 跟另外一个 matrix 做 transpose 相乘， 然后可以确保 $\Sigma $ 是 positive definite 的。这边要强调的点就是说，实际上它不是直接 output 一个 matrix。你再去那个 paper 里面 check 一下它的 trick，它可以保证说 $\Sigma$ 是 positive definite 的。
* 你把 a 代入 $\mu(s)$ 以后，你可以让 Q 的值最大。所以假设要你 arg max 这个东西，虽然一般而言，若 Q 是一个 general function， 你很难算，但是我们这边 design 了 Q 这个 function，a 只要设 $\mu(s)$，我们就得到最大值。你在解这个 arg max 的 problem 的时候就变得非常容易。所以 Q-learning 也可以用在 continuous 的 case，只是有一些局限，就是 function 不能够随便乱设，它必须有一些限制。

# Actor-Critic

**A3C：** Asynchronous（异步的）Advantage Actor-Critic的缩写，一种改进的Actor-Critic方法，通过异步的操作，进行RL模型训练的加速。

**Pathwise Derivative Policy Gradient：** 其为使用 Q-learning 解 continuous action 的方法，也是一种 Actor-Critic 方法。其会对于actor提供value最大的action，而不仅仅是提供某一个action的好坏程度。

## 整个Advantage actor-critic（A2C）算法的工作流程是怎样的？

在传统的方法中，我们有一个policy $\pi$ 以及一个初始的actor与environment去做互动，收集数据以及反馈。通过这些每一步得到的数据与反馈，我们就要进一步更新我们的policy $\pi$ ，通常我们所使用的方式是policy gradient。但是对于actor-critic方法，我们不是直接使用每一步得到的数据和反馈进行policy $\pi$ 的更新，而是使用这些数据进行 estimate value function，这里我们通常使用的算法包括前几个chapters重点介绍的TD和MC等算法以及他们的优化算法。接下来我们再基于value function来更新我们的policy，公式如下：
$$
\nabla \bar{R}_{\theta} \approx \frac{1}{N} \sum_{n=1}^{N} \sum_{t=1}^{T_{n}}\left(r_{t}^{n}+V^{\pi}\left(s_{t+1}^{n}\right)-V^{\pi}\left(s_{t}^{n}\right)\right) \nabla \log p_{\theta}\left(a_{t}^{n} \mid s_{t}^{n}\right)
$$
其中，上式中的 $r_{t}^{n}+V^{\pi}\left(s_{t+1}^{n}\right)-V^{\pi}\left(s_{t}^{n}\right)$ 我们称为Advantage function，我们通过上式得到新的policy后，再去与environment进行交互，然后再重复我们的estimate value function的操作，再用value function来更新我们的policy。以上的整个方法我们称为Advantage Actor-Critic。

## 在实现 Actor-Critic 的时候，有哪些我们用到的tips?


1. **estimate 两个 network：** 一个是estimate V function，另外一个是 policy 的 network，也就是你的 actor。 V-network的input 是一个 state，output 是一个 scalar。然后 actor 这个 network的input 是一个 state，output 是一个 action 的 distribution。这两个 network，actor 和 critic 的 input 都是 s，所以它们前面几个 layer，其实是可以 share 的。尤其是假设你今天是玩 Atari 游戏，input 都是 image。那 input 那个 image 都非常复杂，image 很大，通常前面都会用一些 CNN 来处理，把那些 image 抽象成 high level 的 information，所以对 actor 跟 critic 来说是可以共用的。我们可以让 actor 跟 critic 的前面几个 layer 共用同一组参数。那这一组参数可能是 CNN。先把 input 的 pixel 变成比较 high level 的信息，然后再给 actor 去决定说它要采取什么样的行为，给这个 critic，给 value function 去计算 expected reward。
2. **exploration 机制：** 其目的是对policy $\pi$ 的 output 的分布进行一个限制，从而使得 distribution 的 entropy 不要太小，即希望不同的 action 被采用的机率平均一点。这样在 testing 的时候，它才会多尝试各种不同的 action，才会把这个环境探索的比较好，才会得到比较好的结果。

## 对比经典的Q-learning算法，我们的Pathwise Derivative Policy Gradient有哪些改进之处？


1. 首先，把 $Q(s,a)$ 换成 了 $\pi$，之前是用 $Q(s,a)$ 来决定在 state $s_t$ 产生那一个 action, $a_{t}$ 现在是直接用 $\pi$ 。原先我们需要解 argmax 的问题，现在我们直接训练了一个 actor。这个 actor input $s_t$ 就会告诉我们应该采取哪一个 $a_{t}$。综上，本来 input $s_t$，采取哪一个 $a_t$，是 $Q(s,a)$ 决定的。在 Pathwise Derivative Policy Gradient 里面，我们会直接用 $\pi$ 来决定。
2. 另外，原本是要计算在 $s_{i+1}$ 时对应的 policy 采取的 action a 会得到多少的 Q value，那你会采取让 $\hat{Q}$ 最大的那个 action a。现在因为我们不需要再解argmax 的问题。所以现在我们就直接把 $s_{i+1}$ 代入到 policy $\pi$ 里面，直接就会得到在 $s_{i+1}$ 下，哪一个 action 会给我们最大的 Q value，那你在这边就会 take 那一个 action。在 Q-function 里面，有两个 Q network，一个是真正的 Q network，另外一个是 target Q network。那实际上你在 implement 这个 algorithm 的时候，你也会有两个 actor，你会有一个真正要 learn 的 actor $\pi$，你会有一个 target actor $\hat{\pi}$ 。但现在因为哪一个 action a 可以让 $\hat{Q}$ 最大这件事情已经被用那个 policy 取代掉了，所以我们要知道哪一个 action a 可以让 $\hat{Q}$ 最大，就直接把那个 state 带到 $\hat{\pi}$ 里面，看它得到哪一个 a，就用那一个 a，其也就是会让 $\hat{Q}(s,a)$ 的值最大的那个 a 。
3. 还有，之前只要 learn Q，现在你多 learn 一个 $\pi$，其目的在于maximize Q-function，希望你得到的这个 actor，它可以让你的 Q-function output 越大越好，这个跟 learn GAN 里面的 generator 的概念类似。
4. 最后，与原来的 Q-function 一样。我们要把 target 的 Q-network 取代掉，你现在也要把 target policy 取代掉。